# PUMS Household Income vs. AMI (2021) in Phoenix  PUMAS

- https://www.census.gov/data/developers/data-sets/

For households by income and household size to compare to HUD AMI in same year
-  https://api.census.gov/data/2021/acs/acs1/pums/variables.html

2021 AMI by HH Size (from City of Phoenix)
- https://www.phoenix.gov/humanservicessite/Documents/2021%20AMI%20Limits%204.2.21.pdf

In [1]:
import pandas as pd
import math
import numpy as np
import os

In [2]:
import get_pums as get
import pums as calc

In [3]:
#Search parameters
y1 = '2021'
#y0 = '2013'

sample = 'acs1'

phx_pumas = ['0400112','0400113','0400114','0400115','0400116','0400117',\
             '0400118','0400119','0400120','0400121','0400122','0400123',\
             '0400125','0400128','0400129']

data_cols = 'SERIALNO,ST,PUMA,HINCP,NP,WGTP'

In [4]:
#AMI bands in 2021 by household size
AMI_30pct = {'1':16600,'2':19000,'3':21960,'4':26500,'5':31040,'6':35580,\
            '7':40120,'8':44660}
AMI_50pct = {'1':27650,'2':31600,'3':35500,'4':39500,'5':42700,'6':45850,\
            '7':49000,'8':52150}
AMI_80pct = {'1':44250,'2':50600,'3':56900,'4':63200,'5':68300,'6':73350,\
            '7':78400,'8':83450}
AMI_100pct = {'1':55300,'2':63200,'3':71100,'4':79000,'5':85400,'6':91700,\
            '7':98000,'8':104300}

In [5]:
# create a list of replicate weights
repwt = 'WGTP'
repwts = [repwt+str(i) for i in range(1, 81)]

## Get PUMA data

In [6]:
raw = get.get_puma(sample,y1,data_cols)

In [7]:
df = raw.copy()

In [8]:
df['GEO_ID'] = df['ST']+df['PUMA']
df = df[df.GEO_ID.isin(phx_pumas)]
df  = df.drop(['SERIALNO','ST','PUMA'],axis=1)
df = df[['GEO_ID']+[col for col in df.columns if col != 'GEO_ID']] #move id to first col
for col in df.columns[1:]: df[col] = df[col].astype(int)

In [9]:
df['HHSz'] = pd.cut(df['NP'],bins=[0,1,2,3,4,5,6,7,14],\
                   labels=['1','2','3','4','5','6','7','8'])

In [10]:
df.head()

,GEO_ID,HINCP,NP,WGTP,WGTP1,WGTP2,WGTP3,WGTP4,WGTP5,WGTP6,...,WGTP72,WGTP73,WGTP74,WGTP75,WGTP76,WGTP77,WGTP78,WGTP79,WGTP80,HHSz
4,0400119,-60000,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
33,0400129,-60000,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
38,0400120,-60000,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
40,0400116,-60000,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
46,0400129,-60000,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [11]:
dff = df[~(df.HHSz.isna()) & (df.HINCP!=-60000)].copy()
print(dff.WGTP.sum())
dff['HHSz'] = dff['HHSz'].astype(str)

666238


In [12]:
#assign AMI range based on household size and ami dictionaries
dff['AMI_range'] = np.where((dff['HINCP']<=dff['HHSz'].map(AMI_30pct)),'u30_ami',\
                   np.where((dff['HINCP']>dff['HHSz'].map(AMI_30pct)&dff['HINCP']<=dff['HHSz'].map(AMI_50pct)),'30_50_ami',\
                   np.where((dff['HINCP']>dff['HHSz'].map(AMI_50pct))&(dff['HINCP']<=dff['HHSz'].map(AMI_80pct)),\
                   '50_80_ami',np.where((dff['HINCP']>dff['HHSz'].map(AMI_80pct))&(dff['HINCP']<=dff['HHSz'].map(AMI_100pct)),\
                   '80_100_ami','o100_ami'))))

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [ ]:
pivot = dff.drop(['HINCP','NP','HHSz'],axis=1).copy()
pivot = pivot.groupby(['GEO_ID','AMI_range']).sum().reset_index()
pivot.rename(columns={'WGTP':'hh_e'},inplace=True)

In [ ]:
pivot.head()

In [ ]:
pivot['hh_SE'] = pivot.apply(lambda x: (calc.get_se(x['hh_e'],x[repwts])),axis=1)
pivot['hh_MOE'] = pivot.apply(lambda x: (calc.get_moe(x['hh_SE'])),axis=1)
pivot['hh_CV'] = pivot.apply(lambda x: (calc.get_cv(x['hh_e'],x['hh_SE'])),axis=1)
pivot = pivot.drop(columns=repwts)

In [ ]:
final = pd.pivot_table(pivot,values=['hh_e','hh_MOE','hh_CV'],index='GEO_ID',\
                      columns=['AMI_range'],aggfunc=np.sum).reset_index()

In [ ]:
final

In [ ]:
final.to_excel('output/hhinc_ami_pumas_21.xlsx')

### PUMA - median HHI by HH Size, total HHs by Size

pum_size = get.get_puma('acs/acs1',y1,cols_inc_size)
pum_size = clean_table(pum_size)
pum_size = pum_size[pum_size['GEO_ID'].isin(phx_pumas)]

pumas_rename = {'B19019_001E':'hh_tot_E','B19019_001M':'hh_tot_M','B19019_002E':'hh_1_E','B19019_002M':'hh_1_M',\
               'B19019_003E':'hh_2_E','B19019_003M':'hh_2_M','B19019_004E':'hh_3_E','B19019_004M':'hh_3_M',\
               'B19019_005E':'hh_4_E','B19019_005M':'hh_4_M','B19019_006E':'hh_5_E','B19019_006M':'hh_5_M',\
               'B19019_007E':'hh_6_E','B19019_007M':'hh_6_M','B19019_008E':'hh_7_E','B19019_008M':'hh_7_M'}

pum_size.rename(columns=pumas_rename,inplace=True) 

pum_size = geo_agg.sumgeo_cv(pum_size,'GEO_ID')

pum_size

pum_size.to_excel('output/medhhi_hhsize.xlsx',index=False)